## Load libraries

In [ ]:
import numpy as np
import plotly.express as px
import cupy

## Create toy data

In [ ]:
# Toy data D.
tc = np.random.rand(1,500)
D = np.random.rand(500,100,100)
D[:,0,0] = tc
for i in np.arange(2,2+20):
    for j in np.arange(2,2+20):
        D[:,i,j] = 0 + 0.5*tc + 0.5*np.random.rand(1,500)

In [ ]:
# Show D
fig_D_heatmap = px.imshow(D[3,:,:])
fig_D_heatmap.show()
fig_D_lines = go.Figure()
fig_D_lines.add_trace(go.Scatter(x = np.arange(100), y = D[:,0,0], mode = 'lines', name = '0,0'))
fig_D_lines.add_trace(go.Scatter(x = np.arange(100), y = D[:,2,2], mode = 'lines', name = '2,2'))
fig_D_lines.add_trace(go.Scatter(x = np.arange(100), y = D[:,2,3], mode = 'lines', name = '2,3'))
fig_D_lines.add_trace(go.Scatter(x = np.arange(100), y = D[:,3,2], mode = 'lines', name = '3,2'))
fig_D_lines.add_trace(go.Scatter(x = np.arange(100), y = D[:,3,3], mode = 'lines', name = '3,3'))
fig_D_lines.show()
Dr = np.reshape(D,(500,100*100))
fig_D_lines_reshape = go.Figure()
fig_D_lines_reshape.add_trace(go.Scatter(x = np.arange(100), y = Dr[:,0], mode = 'lines', name = '0,0'))
fig_D_lines_reshape.add_trace(go.Scatter(x = np.arange(100), y = Dr[:,202], mode = 'lines', name = '2,2'))
fig_D_lines_reshape.add_trace(go.Scatter(x = np.arange(100), y = Dr[:,203], mode = 'lines', name = '2,3'))
fig_D_lines_reshape.add_trace(go.Scatter(x = np.arange(100), y = Dr[:,302], mode = 'lines', name = '3,2'))
fig_D_lines_reshape.add_trace(go.Scatter(x = np.arange(100), y = Dr[:,303], mode = 'lines', name = '3,3'))
fig_D_lines_reshape.show()

In [ ]:
D = np.reshape(D,(D.shape[0],D.shape[1]*D.shape[2]))

In [ ]:
# Degree centrality (with GPU)
# Make batch indices
batch = dict()
Y = cupy.asarray(D)

# Set up batches
nbatch = 100 # currently recommend 256.
#nbatch = 2
[i,j]=Y.shape
batch_size=np.intc(j/nbatch)
vox_range=np.array([0])
b=0
while vox_range[-1]+1 <= j:
  if vox_range[-1]+1+batch_size > j:
    vox_range=np.arange(vox_range[-1]+1,j+1)
  else:
    vox_range=np.arange(vox_range[-1]+1,vox_range[-1]+1+batch_size)
  batch.update({b:vox_range-1})
  b += 1

DC = np.zeros(j)
linelength = 1

In [ ]:
def update_out(bcount,batch):
    out.clear_output()
    with out:
        print('Progress: Batch '+str(bcount)+'/'+str(len(batch)**2))

out = widgets.Output()
out

Output()

In [ ]:
# Loop through batches
bcount = 1

for b_col in np.arange(len(batch)):
    # Compute correlations
    R = cupy.zeros([j,batch_size])
    for b_row in np.arange(len(batch)):
        update_out(bcount,batch)
        bcount += 1
        R[batch[b_row],:] = cupy.corrcoef(Y[:,batch[b_row]],Y[:,batch[b_col]],rowvar=False)[len(batch[b_row]):len(batch[b_row])*2,0:len(batch[b_row])].T
    I = np.concatenate((np.zeros([batch_size*(b_col),batch_size]),np.eye(batch_size),np.zeros([batch_size*(nbatch-b_col-1),batch_size])),axis=0)==1
    R[I] = cupy.nan
    DC[batch[b_col]] = cupy.asnumpy(cupy.nanmean(cupy.arctanh(R),axis=0))

In [ ]:
eDC = np.reshape(DC,(100,100))
fig1 = px.imshow(eDC,range_color=(0.015,0.05))
#fig1 = px.imshow(eDC)
fig1.show()

In [ ]:
eD = np.reshape(np.sum(D,axis=0),(100,100))
#fig_eD = px.imshow(eD,range_color=(50000,100000))
fig_eD = px.imshow(eD)
fig_eD.show()